In [1]:
import sqlite3
import numpy as np
import pandas as pd
from datetime import datetime as dt, timedelta

In [3]:
airlines_df = pd.read_excel("../data/Airlines.xlsx", index_col="airline_id")
airports_df = pd.read_excel("../data/Airports.xlsx", index_col="airport_id")
routes_df = pd.read_excel("../data/Routes.xlsx", index_col="route_id")
flights_df = pd.read_excel("../data/Flights.xlsx", index_col="flight_id")

In [4]:
INSERT_TEMPLATE = "INSERT INTO {table_name} {columns_tuple} VALUES {values_tuple};\n"

COLUMNS_INSERT_TEMPLATE = "INSERT INTO {table_name} {columns_tuple}"
VALUE_INSERT_TEMPLATE = "VALUES {values_tuple};\n"

In [5]:
def insert_into_sql_table_sqlite(data, table_name, index):
    columns = (data.index.name, ) + tuple(data.columns)
    values = (index, ) + tuple(data.iloc[index - 1].tolist())
    
    return INSERT_TEMPLATE.format(table_name=table_name, columns_tuple=columns, values_tuple=values).replace("\'", "\"")

In [6]:
def insert_into_sql_table_postgre(data, table_name, index):
    columns = (data.index.name, ) + tuple(data.columns)
    columns = tuple([column.lower().replace("\'", "") for column in columns])
    values = (index, ) + tuple(data.iloc[index - 1].tolist())
    
    return COLUMNS_INSERT_TEMPLATE.format(table_name=table_name.lower(), columns_tuple=columns).replace("\'", "") + " " + VALUE_INSERT_TEMPLATE.format(values_tuple=values)

In [7]:
insert_into_sql_table_postgre(airlines_df, "Airline", 4)

"INSERT INTO airline (airline_id, name, airline_code) VALUES (4, 'Air France-KLM', 'SEU');\n"

In [8]:
with open(r"../scripts/sql/populate_db.sql", "w", encoding="utf8") as f:
    for index in airlines_df.index.tolist():
        f.write(insert_into_sql_table_postgre(airlines_df, "Airline", index))
    f.write("\n")
        
    for index in airports_df.index.tolist():
        f.write(insert_into_sql_table_postgre(airports_df, "Airport", index))
    f.write("\n")
        
    for index in routes_df.index.tolist():
        f.write(insert_into_sql_table_postgre(routes_df, "Route", index))
    f.write("\n")
    
    for index in flights_df.index.tolist():
        f.write(insert_into_sql_table_postgre(flights_df, "Flight", index))
    f.write("\n")